In [1]:
! pip install flask-ngrok

<h3> Libraries </h3>

In [2]:
import pandas as pd
import numpy as np

from scipy.sparse import hstack
from google.colab import files
from sklearn.calibration import CalibratedClassifierCV
import tqdm.notebook as tq
from xgboost  import XGBClassifier
from joblib import dump, load
import warnings
warnings.filterwarnings("ignore")

<h3> Preprocessing </h3>

In [3]:
uploaded=files.upload()

Saving bank-additional-full.csv to bank-additional-full.csv


In [17]:
data=pd.read_csv('bank-additional-full.csv',sep=";")

<h5>1. Converting Age to Age_Group </h5>

In [12]:
#Reference https://www.absentdata.com/pandas/pandas-cut-continuous-to-categorical/
def age_to_categorical(data1,age):
    age_group=pd.cut(age,bins=[10,20,30,40,50,60,70,80,90,100],labels=['10-19','20-29','30-39','40-49','50-59','60-69','70-79','80-89','90-100'])
    #inserting the age group after age and deleting it
    data1.insert(1,'age_group',age_group)
    #dropping age column
    data1.drop('age',axis=1,inplace=True)
    
    return data1
   


<h5> 2. Creating I-Loan from Personal and Housing Loan </h5>

In [13]:
#creating i_loan based on the columns['loan','housing'] status we have 3 status yes,no, unknown and if any 2 columns have status yes then
#i_loan will have yes as status, if none of them have yes as status but no as status then i_loan will have no status else unknown status.
#dropped the housing,loan column and added i_loan column. 
def creating_i_loan(data2):
    i_loan=[]
    for index in data2.index.tolist():
        if (data2['housing'].loc[index]=='yes' or data2['loan'].loc[index]=='yes'):
            i_loan.append('yes')
        elif (data2['housing'].loc[index]=='no' or data2['loan'].loc[index]=='no'):
            i_loan.append('no')
        
        else:
            i_loan.append('unknown')
    data2.insert(5,'i_loan',i_loan,True)   
    data2.drop(['loan','housing'],axis=1,inplace=True)
    return data2           



<h5>3. Response Encoding </h5>

In [14]:
"""this function is used to transform our dataset based on our fit train dataset. If the category is present it will replace the it with
positive and negative probablity computed during fit function if category is not present it would it with 0.5 probablity for both
positive and negative."""
def transform_response_encoding(feature,positive,negative):
    unique_list=np.unique(feature).tolist()
   
    
    
    negative_test={}
    positive_test={}
    for value in unique_list:
        #value which is not present in training data probablity values would be 0.5
        
        if value not in positive.keys() and negative.keys():
           
            negative_test[value]=0.5
            positive_test[value]=0.5
        else:
            negative_test[value]=negative[value]
            positive_test[value]=positive[value]
    return positive_test,negative_test

In [15]:
"""In this function I have passed my dataset in which we have to transform along with the dictionary
and returning the dataframe"""
def transform_encoding_of_features(data,dictionary):
    new_data=pd.DataFrame()
    
    for feature in ['age_group', 'job', 'marital', 'education', 'default', 'i_loan', 'contact', 'month', 'day_of_week', 'poutcome']:
        positive_values,negative_values=transform_response_encoding(data[feature],dictionary[feature][0],dictionary[feature][1])
        variable1=feature+'_'+'positive'
        variable2=feature+'_'+'negative'
        new_data[variable1]=data[feature].map(positive_values)
        new_data[variable2]=data[feature].map(negative_values)
    return new_data   



<h5> 4. Merging Categorical Columns with Numerical Columns </h5>

In [16]:
"""In this function I am merging my all numerical data with corresponding dataset. Here I have not included pdays and duration category"""
def merging_columns(new_dataset,dataset):
    data=new_dataset
    for columns in ['campaign','previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']:
        data=pd.concat([data,dataset[columns]],axis=1)
    return data  

<h3> Final Function for Model Deployment </h3>

In [26]:
"""This function takes input, converts it into DataFrame,loads the model,load the dictionary of response
encoding, does preprocessing,and return the prediction"""
def final(X):
    X=X.to_frame().transpose()
    dictionary=load('/content/response_encoding.joblib')
    model=load('/content/bestmodel.joblib')
    age_to_categorical(X,X['age'])
    creating_i_loan(X)
    response=transform_encoding_of_features(X,dictionary)
    merged=merging_columns(response,X)
  
    y_predictions=model.predict(merged)
    return y_predictions






In [27]:
final(data.loc[0])

array([0])

<h3> Model Deployment </h3>

In [ ]:
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok 
app = Flask(__name__,template_folder='/content')
run_with_ngrok(app)    
  
@app.route("/") 
def home(): 
    return render_template("/1.html")
@app.route('/result',methods = ['POST', 'GET'])
def result():
    #loading the dictionary for response encoding.
    dictionary=load('/content/response_encoding.joblib')
    #loading the model to do predcitions.
    model=load('/content/bestmodel.joblib')
    #appending the values received from the list 
    l=[]
    age=int(request.form.get('age'))
    l.append(age)
    l.append(request.form.get('job'))
    l.append(request.form.get('marital'))
    l.append(request.form.get('education'))
    l.append(request.form.get('default'))
    l.append(request.form.get('housing'))
    l.append(request.form.get('loan'))
    l.append(request.form.get('contact'))
    l.append(request.form.get('month'))
    l.append(request.form.get('day_of_week'))
    l.append(request.form.get('poutcome'))
    campaign=int(request.form.get('campaign'))    
    l.append(campaign)
    pdays=int(request.form.get('pdays'))  
    l.append(pdays)
    previous=int(request.form.get('previous'))  
    l.append(previous)
    emp=float(request.form.get('emp.var.rate'))  
    l.append(emp)
    cons=float(request.form.get('cons.price.idx'))  
    l.append(cons)
    conf=float(request.form.get('cons.conf.idx')) 
    l.append(conf)
    euribor=float(request.form.get('euribor3m')) 
    l.append(euribor)
    nr=float(request.form.get('nr.employed')) 
    l.append(nr)
    #creating the dataframe to do preprocessing required for our model to predict
    df=pd.DataFrame([l],columns=request.form.to_dict().keys())
    #preprocessing 
    age_to_categorical(df,df['age'])
    creating_i_loan(df)
    response=transform_encoding_of_features(df,dictionary)
    merged=merging_columns(response,df)
    #predictions from the model.
    y_predictions=model.predict(merged)
    if y_predictions==[0]:
        return "Client will not subscibe for deposits"
    else:
        return "Client will subscribe for deposits"    
   
 
    

 

   
 
    
    
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://463214464002.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [27/Feb/2021 09:12:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Feb/2021 09:12:51] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [27/Feb/2021 09:14:21] "POST /result HTTP/1.1" 200 -


My Mode Deployement Video [Click Here](https://youtu.be/220fqul83zs)